In [2]:
import torch
import torch.nn as nn
from models.classifier import CustomClassifier
from models.model import UnimodalModel
from encoders.image_encoders import DenseNet3D, ResNet50 , VIT
from encoders.tabular_encoders import TabEncoderMLP, ClinicalBERTEncoder
from datasets import MMClassificationCollator
from transformers import AutoImageProcessor
from preprocessing.MRI_processing import GetMiddleSlice
from torch.nn.modules.utils import consume_prefix_in_state_dict_if_present

from preprocessing.text_processing import get_serialize_data_processing
from datasets import get_adni_dataset_new, data_loader, MMClassificationCollator
from monai.transforms import (
    Compose,
    LoadImage,
    ScaleIntensity,
    EnsureChannelFirst,
    Resize,
    EnsureType
)

In [2]:
torch.cuda.is_available()

True

# Data loading

In [3]:
seed = 42
yaml_db_path="ADNI_dataset.yaml"
import yaml

with open(yaml_db_path, 'r') as f:
    dataset_config = yaml.safe_load(f)
print(dataset_config)


class SimpleConfig:
        def __init__(self):
            self.batch_size = 1 
            self.num_workers = 4
            self.pin_memory = True
            self.persistent_workers = True
            self.dataset = dataset_config
            
        class model:
            weight_samples = False
    
cfg = SimpleConfig()

dataset_args = dict(dataset_config)


X_train, y_train, X_val, y_val, X_test, y_test, class_names, le = get_adni_dataset_new(**dataset_args,seed=seed)


# Set up the transforms pipeline
transforms = {
    "transform_fun": Compose([
        LoadImage(image_only=True),
        ScaleIntensity(),
        EnsureChannelFirst(),
        Resize(spatial_size=[128, 128, 128]),
        EnsureType()
    ]),
    "params": {},
    "data_type": "nii.gz"
}


collate_fn = MMClassificationCollator( 
    image_processing=transforms,
    tabular_processing=None)


test_dataloader = data_loader(X_test, y_test,collate_fn=collate_fn, config=cfg, shuffle=True)


{'dataset_name': 'ADNI MRI dataset New', 'data_path': '/gpfs/projects/acad/mmfusion/datasets/adni/ADNIDenoise/AD_CN_clinical_duplicates_aligned.csv', 'image_column': 'nii_path', 'image_type': 'nii.gz', 'eeg_column': None, 'text_columns': None, 'target_column': 'Group', 'subject_column': 'Subject', 'train_val_size': 0.1, 'train_test_size': 0.1, 'normalize': True, 'make_dummies': False, 'drop_duplicates': False, 'patient_based_split': True, 'nunique_cat_values': [2, 2, 5, 2, 7, 2, 3, 3, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], 'cat_cols': ['PTRACCAT', 'PTGENDER', 'PTHAND', 'PTMARRY', 'PTNOTRT', 'PTTLANG', 'MH14ALCH', 'MH17MALI', 'MH16SMOK', 'MH15DRUG', 'MH4CARD', 'MHPSYCH', 'MH2NEURL', 'MH6HEPAT', 'MH12RENA', 'DSPANFOR', 'DSPANBAC', 'CDGLOBAL', 'BCFAQ', 'BCDEPRES'], 'cat_cols_dummies': ['PTRACCAT_2', 'PTRACCAT_4', 'PTRACCAT_5', 'PTRACCAT_6', 'PTGENDER_1.0', 'PTGENDER_2.0', 'PTHAND_1.0', 'PTHAND_2.0', 'PTMARRY_1.0', 'PTMARRY_2.0', 'PTMARRY_3.0', 'PTMARRY_4.0', 'PTMARRY_5.0', 'PTNOTRT_0.

# 3D MRI models 

In [ ]:
densenet3d_chkpt_patient = "best_models/best_model_densenet3D_epoch=14-step=3510.ckpt"

image_encoder = DenseNet3D(
            freeze=False,
            include_head=False
          )

classifier = CustomClassifier(
    hidden_dim=0,
    activation_fun=nn.ReLU(),
    num_class=2, 
    task="multiclass",  
    dropout_rate=0
)

encoders = {"image": image_encoder}
unimodal_model = UnimodalModel(
    encoders=encoders,
    classifier=classifier
)
dummy_input = next(iter(test_dataloader))

unimodal_model(dummy_input)
ckpt = torch.load(densenet3d_chkpt_patient, map_location="cpu")
state_dict = ckpt["state_dict"]
consume_prefix_in_state_dict_if_present(state_dict, "model.")
unimodal_model.load_state_dict(state_dict)

## Generate embedding

In [ ]:
import torch, pickle
import numpy as np
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
unimodal_model.to(device)
unimodal_model.eval()


label_mapping ={0: "AD", 1: "CN"}


records_by_subject = {}
records_by_class = {}

offset = 0
with torch.no_grad():
    for i,batch in enumerate(test_dataloader):
        X_test_row = X_test.iloc[i]
        # move tensors to device
        for k, v in batch.items():
            if torch.is_tensor(v):
                batch[k] = v.to(device)

        # get labels, embeddings, subjects
        label = batch["label"].view(-1).long()
        embs = unimodal_model.encoders["image"](batch["image"])  # shape [B, D]
        subject_id = X_test_row["Subject"]
        class_name = label_mapping[int(label.item())]
        emb_np = embs.detach().cpu().numpy().astype(np.float32)

        records_by_subject[subject_id] = {"class_name": class_name, "embedding": emb_np}
out_pkl = "test_embeddings_per_patient.pkl"
with open(out_pkl, "wb") as f:
    pickle.dump(records_by_subject, f, protocol=pickle.HIGHEST_PROTOCOL)

print(f"Saved: {out_pkl}")
print(f"Subjects: {len(records_by_subject)} | Dims: {emb_np.shape}")

Saved: test_embeddings_per_patient.pkl
Subjects: 33 | Dims: (1, 1024)


## Prediction loop

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
unimodal_model.to(device)
unimodal_model.eval()

criterion = nn.CrossEntropyLoss()

total_loss = 0.0
total = 0
correct = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        for key in batch.keys():
            batch[key] = batch[key].to(device)        # get labels
        label_key = "label"
        targets = batch[label_key].view(-1).long()

        # forward
        logits = unimodal_model(batch)

        # loss
        loss = criterion(logits, targets)
        total_loss += loss.item() * targets.size(0)

        # predictions
        preds = torch.argmax(logits, dim=1)

        correct += (preds == targets).sum().item()
        total += targets.size(0)

        all_preds.append(preds.detach().cpu())
        all_labels.append(targets.detach().cpu())

avg_loss = total_loss / total if total > 0 else float("nan")
accuracy = correct / total if total > 0 else float("nan")

print(f"Test Loss (CE): {avg_loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

Test Loss (CE): 0.1507
Test Accuracy: 94.44%


# 2D MRI models Resnet50 (coronal View)

## Data Loading

In [9]:
# 1. Set up the transforms
transforms = {
    "transform_fun": Compose([
        GetMiddleSlice(axis="Coronal"),
        ScaleIntensity(),
        EnsureType()
    ]),
    "params": {},
    "data_type": "nii.gz"
}

collate_fn = MMClassificationCollator( 
    image_processing=transforms,
    tabular_processing=None)


test_dataloader = data_loader(X_test, y_test,collate_fn=collate_fn, config=cfg, shuffle=False)




## Model Loading

In [ ]:
resnet50_ckpt = "best_models/best_model_resnet50_epoch=9-step=590.ckpt"

image_encoder = ResNet50(
            freeze=False,
            include_head=False
          )

classifier = CustomClassifier(
    hidden_dim=0,  
    activation_fun=nn.ReLU(),
    num_class=2, 
    task="multiclass", 
    dropout_rate=0
)

encoders = {"image": image_encoder}
unimodal_model = UnimodalModel(
    encoders=encoders,
    classifier=classifier
)


dummy_input = next(iter(test_dataloader))

unimodal_model(dummy_input)

ckpt = torch.load(resnet50_ckpt, map_location="cpu")
state_dict = ckpt["state_dict"]
consume_prefix_in_state_dict_if_present(state_dict, "model.")
unimodal_model.load_state_dict(state_dict)


Initializing layers with input dimension: 2048


/tmp/ipykernel_75442/2984451003.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(resnet50_ckpt, map_location="cpu")


<All keys matched successfully>

# 2D MRI Model VIT (coronal view)

## Data Loading

In [3]:
cfg.batch_size = 2
# 1. Set up the transforms
transforms = {
    "transform_fun": Compose([
        GetMiddleSlice(axis="Coronal"),
        ScaleIntensity(),
        EnsureType(),
        AutoImageProcessor.from_pretrained(
            pretrained_model_name_or_path="google/vit-base-patch16-224-in21k",
            do_rescale=False,
            use_fast=True,
            return_tensors="pt"
        )
    ]),
    "params": {},
    "data_type": "nii.gz"
}

collate_fn = MMClassificationCollator( 
    image_processing=transforms,
    tabular_processing=None)


test_dataloader = data_loader(X_test, y_test,collate_fn=collate_fn, config=cfg, shuffle=False)



## Model Loading

In [ ]:
vit_model_ckpt="best_models/best_model_VIT_epoch=4-step=295.ckpt"

image_encoder = VIT(
            freeze=False,
            include_head=False
          )

classifier = CustomClassifier(
    hidden_dim=0,  
    activation_fun=nn.ReLU(),
    num_class=2,  
    task="multiclass",  
    dropout_rate=0
)

encoders = {"image": image_encoder}
unimodal_model = UnimodalModel(
    encoders=encoders,
    classifier=classifier
)

dummy_input = next(iter(test_dataloader))

unimodal_model(dummy_input)

ckpt = torch.load(vit_model_ckpt, map_location="cpu")
state_dict = ckpt["state_dict"]
consume_prefix_in_state_dict_if_present(state_dict, "model.")
unimodal_model.load_state_dict(state_dict)

Initializing layers with input dimension: 768


/tmp/ipykernel_84004/3783239972.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(vit_model_ckpt, map_location="cpu")


<All keys matched successfully>

# Tabular modality Model (MLP)

## Data Loading

In [9]:
seed = 42
yaml_db_path="ADNI_dataset.yaml"
import yaml

with open(yaml_db_path, 'r') as f:
    dataset_config = yaml.safe_load(f)

dataset_config["drop_duplicates"] = True
class SimpleConfig:
        def __init__(self):
            self.batch_size = 4 
            self.num_workers = 4
            self.pin_memory = True
            self.persistent_workers = True
            self.dataset = dataset_config
            
        class model:
            weight_samples = False
    
cfg = SimpleConfig()

# Load dataset using the same pattern as data_utils.py
dataset_args = dict(dataset_config)


X_train, y_train, X_val, y_val, X_test, y_test, class_names, le = get_adni_dataset_new(**dataset_args,seed=seed)



collate_fn = MMClassificationCollator( 
    image_processing=None,
    tabular_processing=True)


test_dataloader = data_loader(X_test, y_test,collate_fn=collate_fn, config=cfg, shuffle=True)

## Model Loading

In [ ]:
model_ckpt= "best_models/best_model_MLP_epoch=9-step=170-v2.ckpt"


tab_encoder = TabEncoderMLP(
            freeze=False,
            include_head=False,
          )

classifier = CustomClassifier(
    hidden_dim=0,  
    activation_fun=nn.ReLU(),
    num_class=2,  
    task="multiclass",  
    dropout_rate=0
)

encoders = {"tabular": tab_encoder}
unimodal_model = FusionModel(
    encoders=encoders,
    fusion=None,  # since we're only using MRIs only
    classifier=classifier
)

dummy_input = next(iter(test_dataloader))

unimodal_model(dummy_input)

ckpt = torch.load(model_ckpt, map_location="cpu")
state_dict = ckpt["state_dict"]
consume_prefix_in_state_dict_if_present(state_dict, "model.")
unimodal_model.load_state_dict(state_dict)

Initializing layers with input dimension: 32


/tmp/ipykernel_84004/2903487734.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_ckpt, map_location="cpu")


<All keys matched successfully>

# Tabular Model ClinicalBert (best for tabular modality)

## Data Loading

In [38]:
seed = 42
yaml_db_path="ADNI_dataset.yaml"
import yaml

with open(yaml_db_path, 'r') as f:
    dataset_config = yaml.safe_load(f)

dataset_config["drop_duplicates"] = True
dataset_config["normalize"] = True

class SimpleConfig:
        def __init__(self):
            self.batch_size = 1 
            self.num_workers = 4
            self.pin_memory = False
            self.persistent_workers = True
            self.dataset = dataset_config
            
        class model:
            weight_samples = False
    
cfg = SimpleConfig()

# Load dataset using the same pattern as data_utils.py
dataset_args = dict(dataset_config)


X_train, y_train, X_val, y_val, X_test, y_test, class_names, le = get_adni_dataset_new(**dataset_args,seed=seed)


collate_fn = MMClassificationCollator( 
    image_processing=None,
    tabular_processing=get_serialize_data_processing())


test_dataloader = data_loader(X_test, y_test,collate_fn=collate_fn, config=cfg, shuffle=True)

## Model Loading

In [ ]:
#model_ckpt = "best_models/best_model_clinicalBert_epoch=7-step=72-v1.ckpt"
model_ckpt = "best_models/best_model_clinicalBert_epoch=7-step=72-v1.ckpt"

tab_encoder = ClinicalBERTEncoder(
            freeze=False,
            include_head=False,
            checkpoint="medicalai/ClinicalBERT"
          )

classifier = CustomClassifier(
    hidden_dim=0,  
    activation_fun=nn.ReLU(),
    num_class=2,  
    task="multiclass",  
    dropout_rate=0
)

encoders = {"tabular": tab_encoder}
unimodal_model = UnimodalModel(
    encoders=encoders,
    classifier=classifier
)

dummy_input = next(iter(test_dataloader))

unimodal_model(dummy_input)

ckpt = torch.load(model_ckpt, map_location="cpu")
state_dict = ckpt["state_dict"]
consume_prefix_in_state_dict_if_present(state_dict, "model.")
unimodal_model.load_state_dict(state_dict)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/tmp/ipykernel_84004/2782077999.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub fo

Initializing layers with input dimension: 768


<All keys matched successfully>